In [1]:

# imports
import os
import sys
import types
import json

# figure size/format
fig_width = 7
fig_height = 5
fig_format = 'retina'
fig_dpi = 96

# matplotlib defaults / format
try:
  import matplotlib.pyplot as plt
  plt.rcParams['figure.figsize'] = (fig_width, fig_height)
  plt.rcParams['figure.dpi'] = fig_dpi
  plt.rcParams['savefig.dpi'] = fig_dpi
  from IPython.display import set_matplotlib_formats
  set_matplotlib_formats(fig_format)
except Exception:
  pass

# plotly use connected mode
try:
  import plotly.io as pio
  pio.renderers.default = "notebook_connected"
except Exception:
  pass

# enable pandas latex repr when targeting pdfs
try:
  import pandas as pd
  if fig_format == 'pdf':
    pd.set_option('display.latex.repr', True)
except Exception:
  pass



# output kernel dependencies
kernel_deps = dict()
for module in list(sys.modules.values()):
  # Some modules play games with sys.modules (e.g. email/__init__.py
  # in the standard library), and occasionally this can cause strange
  # failures in getattr.  Just ignore anything that's not an ordinary
  # module.
  if not isinstance(module, types.ModuleType):
    continue
  path = getattr(module, "__file__", None)
  if not path:
    continue
  if path.endswith(".pyc") or path.endswith(".pyo"):
    path = path[:-1]
  if not os.path.exists(path):
    continue
  kernel_deps[path] = os.stat(path).st_mtime
print(json.dumps(kernel_deps))

# set run_path if requested
if r'C:\Users\adamt\mathreason\prop_log':
  os.chdir(r'C:\Users\adamt\mathreason\prop_log')

# reset state
%reset

def ojs_define(**kwargs):
  import json
  try:
    # IPython 7.14 preferred import
    from IPython.display import display, HTML
  except:
    from IPython.core.display import display, HTML

  # do some minor magic for convenience when handling pandas
  # dataframes
  def convert(v):
    try:
      import pandas as pd
    except ModuleNotFoundError: # don't do the magic when pandas is not available
      return v
    if type(v) == pd.Series:
      v = pd.DataFrame(v)
    if type(v) == pd.DataFrame:
      j = json.loads(v.T.to_json(orient='split'))
      return dict((k,v) for (k,v) in zip(j["index"], j["data"]))
    else:
      return v
  
  v = dict(contents=list(dict(name=key, value=convert(value)) for (key, value) in kwargs.items()))
  display(HTML('<script type="ojs-define">' + json.dumps(v) + '</script>'), metadata=dict(ojs_define = True))
globals()["ojs_define"] = ojs_define


{"C:\\Program Files\\Python311\\Lib\\importlib\\_bootstrap.py": 1686132018.0, "C:\\Program Files\\Python311\\Lib\\importlib\\_bootstrap_external.py": 1686132018.0, "C:\\Program Files\\Python311\\Lib\\zipimport.py": 1686132018.0, "C:\\Program Files\\Python311\\Lib\\codecs.py": 1686132018.0, "C:\\Program Files\\Python311\\Lib\\encodings\\aliases.py": 1686132018.0, "C:\\Program Files\\Python311\\Lib\\encodings\\__init__.py": 1686132018.0, "C:\\Program Files\\Python311\\Lib\\encodings\\utf_8.py": 1686132018.0, "C:\\Program Files\\Python311\\Lib\\encodings\\cp1252.py": 1686132018.0, "C:\\Program Files\\Python311\\Lib\\abc.py": 1686132018.0, "C:\\Program Files\\Python311\\Lib\\io.py": 1686132018.0, "C:\\Program Files\\Python311\\Lib\\stat.py": 1686132018.0, "C:\\Program Files\\Python311\\Lib\\_collections_abc.py": 1686132018.0, "C:\\Program Files\\Python311\\Lib\\genericpath.py": 1686132018.0, "C:\\Program Files\\Python311\\Lib\\ntpath.py": 1686132018.0, "C:\\Program Files\\Python311\\Lib\\o

C:\Users\adamt\AppData\Local\Temp\ipykernel_4680\1333358119.py:20: DeprecationWarning: `set_matplotlib_formats` is deprecated since IPython 7.23, directly use `matplotlib_inline.backend_inline.set_matplotlib_formats()`
  set_matplotlib_formats(fig_format)


In [2]:
print("1 and 1 = " + str(1 and 1))
print("1 and 0 = " + str(1 and 0))

1 and 1 = 1
1 and 0 = 0


In [3]:
import pyparsing

class Proposition:
    pass

class PropVariable(Proposition):
    def __init__(self, v):
        assert (type(v) == type("") and len(v) == 1) 
        self.v = v
    def __str__(self):
        return(self.v)
    def eval(self, prop_eval):
        return prop_eval(self.v)

class Negation(Proposition):
    def __init__(self, beta):
        assert issubclass(type(beta),Proposition)
        self.neg = beta
    def __str__(self):
        return("(not " + str(self.neg) + ")")
    def eval(self, prop_eval):
        return(1 - self.neg.eval(prop_eval))

class Disjunction(Proposition):
    def __init__(self, beta, gamma):
        assert issubclass(type(beta),Proposition) \
            and issubclass(type(gamma),Proposition)
        self.left, self.right = beta, gamma
    def __str__(self):
        return("(" + str(self.left )+" or " + str(self.right) + ")")
    def eval(self, prop_eval):
        l, r = self.left.eval(prop_eval), self.right.eval(prop_eval)
        return( l + r - l*r )

def parseTree(p):
    if type(p) == type(""):
        return PropVariable(p)
    if len(p) == 1:
        return parseTree(p[0])
    if len(p) == 2:
        assert( p[0] == "not" )
        return Negation(parseTree(p[1]))
    
    if len(p) == 3:
        if p[1] == "or": return Disjunction(parseTree(p[0]), parseTree(p[2]))
        raise Exception("length 3 list but unrecognized middle token")
    raise Exception("length of list unrecognized")

terms = pyparsing.Word(pyparsing.alphas) | "not" | "and" | "or" | "if" | "then" | "iff" 
nesting = pyparsing.nestedExpr( '(', ')', content=terms )

def shape(s): 
    p = nesting.parseString("("+s+")")[0]
    
    return parseTree(p)

In [4]:
def vareval(pv):
    return pv == "P"

In [5]:
def eval(prop_string, prop_eval):
    return bool((shape(prop_string)).eval(prop_eval))

In [6]:
prop_string = "Q or (not (not P))"

print(eval(prop_string,vareval))

print(eval(prop_string, lambda x: False))

True
False
